In [1]:
import numpy as np 
import pandas as pd 

In [2]:
train=pd.read_csv('train.csv')

In [3]:
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


## Features Engineering(Label Encoding) & Exploratory data analysis

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(train['Activity'])

train['Activity_Num']=pd.DataFrame(encoder.transform(train['Activity']))

In [5]:
LL=list(train.columns)
LL.remove('Activity')
corr=[]
for i in range(0,len(LL)):
    A=np.corrcoef(train[LL[i]],train[LL[i]])
    corr.append(float(A[0][1]))

In [6]:
LL=list(train.columns)
LL.remove('Activity')
L=[]
for i in range(1,len(LL)):
    A=train.loc[:,['Activity_Num',LL[i]]].corr()
    if abs(A.iloc[0,1])>=0.8:
        L.append(LL[i])

In [7]:
Train=train.loc[:,L]
df=Train.corr()
L_1=list(df.columns)
L_1.remove('Activity_Num')
A=df.loc[L_1,'Activity_Num']
A=A.nlargest(n=5)
feature=list(A.index)

In [8]:
feature.append(df[df['Activity_Num']<0].index[0])

In [9]:
feature


['fBodyAccJerk-entropy()-X',
 'tBodyGyroJerk-entropy()-Z',
 'tBodyAccJerk-entropy()-Y',
 'tBodyAcc-sma()',
 'tBodyAccJerkMag-entropy()',
 'tBodyAcc-min()-X']

In [10]:
feature.append('Activity_Num')
Train_final=train.loc[:,feature]

In [11]:
feature.append('Activity')
Train_final=train.loc[:,feature]

In [12]:
df=Train_final.loc[:,['Activity_Num','fBodyAccJerk-entropy()-X']]
df.groupby('Activity_Num').mean()

,fBodyAccJerk-entropy()-X
Activity_Num,
0,-0.935105
1,-0.952443
2,-0.930223
3,0.510943
4,0.671334
5,0.448532


In [13]:
df=Train_final.loc[:,['Activity_Num','tBodyAcc-sma()']]
df.groupby('Activity_Num').mean()

,tBodyAcc-sma()
Activity_Num,
0,-0.933084
1,-0.954649
2,-0.957398
3,-0.163050
4,0.114600
5,-0.077195


In [14]:
df=Train_final.loc[:,['Activity_Num','tBodyGyroJerk-entropy()-Z']]
df.groupby('Activity_Num').mean()


,tBodyGyroJerk-entropy()-Z
Activity_Num,
0,-0.581227
1,-0.532745
2,-0.472012
3,0.584251
4,0.609097
5,0.578221


In [15]:
Train_final.reset_index(inplace=True)
df=Train_final.loc[:,['Activity','index']]
df.groupby('Activity').count()

,index
Activity,
LAYING,1407
SITTING,1286
STANDING,1374
WALKING,1226
WALKING_DOWNSTAIRS,986
WALKING_UPSTAIRS,1073


## Building models:
The building of the models would be by combining multiple models using the vote method 

Step 1: Build KNeighborsClassifier using a GridSearch for a better tuning of hyperparameter

Step 2: Using this model to build a bagging classifier

Step 3: Build a Gradient Boosting Classifier 

Step 4: Build a Random Forest Classifer 

Then use a votingClassifer.



## Step 1

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
X=Train_final.loc[:,['fBodyAccJerk-entropy()-X','tBodyGyroJerk-entropy()-Z','tBodyAccJerk-entropy()-Y','tBodyAcc-sma()','tBodyAccJerkMag-entropy()','tBodyAcc-min()-X']]
y=Train_final['Activity_Num']
X_train,X_test,y_train,y_test= train_test_split(X,y)

scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)




model=KNeighborsClassifier().fit(np.array(X_train_scaled),np.array(y_train))

y_train_predicted=model.predict(X_train_scaled)




param_grid={'n_neighbors':[3,4,5,6,7,8,9,10]}





model=GridSearchCV(KNeighborsClassifier(), param_grid,scoring='roc_auc_ovo')

model.fit(X_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc_ovo')

In [17]:
model.best_params_

{'n_neighbors': 10}

In [18]:
model.best_score_

0.9069975135080813

In [19]:
from sklearn.metrics import roc_auc_score
model_1=KNeighborsClassifier(n_neighbors=10).fit(X_train,y_train)
y_predict_train=model_1.predict_proba(X_train)
y_predict_test=model_1.predict_proba(X_test)

roc_auc_score(y_train,y_predict_train,multi_class='ovo'),roc_auc_score(y_test,y_predict_test,multi_class='ovo')


(0.9457477543864198, 0.9130074849451136)

## Step 2

In [20]:
from sklearn.ensemble import BaggingClassifier
clf=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=9))


param_grid={'n_estimators':[40,50,60,70,80]}





model=GridSearchCV(clf,param_grid,scoring='roc_auc_ovo')

model.fit(X_train,y_train)
model.best_params_,model.best_score_

({'n_estimators': 60}, 0.9140802043858789)

In [21]:
model_1_1=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=9),n_estimators=80).fit(X_train,y_train)
y_predict_train=model_1_1.predict_proba(X_train)
y_predict_test=model_1_1.predict_proba(X_test)

roc_auc_score(y_train,y_predict_train,multi_class='ovo'),roc_auc_score(y_test,y_predict_test,multi_class='ovo')

(0.9516261786184393, 0.9187146880388408)

## Step 3

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid={'n_estimators':[40,50,60,70,80],'max_depth':np.arange(3,9)}
model=GridSearchCV(GradientBoostingClassifier(),param_grid,scoring='roc_auc_ovo')




model.fit(X_train,y_train)

model.best_params_,model.best_score_

({'max_depth': 4, 'n_estimators': 80}, 0.9404613801920443)

In [23]:
model_2=GradientBoostingClassifier(n_estimators=80,max_depth=6).fit(X_train,y_train)
y_predict_train=model_2.predict_proba(X_train)
y_predict_test=model_2.predict_proba(X_test)

roc_auc_score(y_train,y_predict_train,multi_class='ovo'),roc_auc_score(y_test,y_predict_test,multi_class='ovo')

(0.9984743524591492, 0.9438336877396255)

## Step 4

In [24]:
from sklearn.ensemble import RandomForestClassifier
model_3=RandomForestClassifier(n_estimators=50,max_depth=6).fit(X_train,y_train)
y_predict_train=model_3.predict_proba(X_train)
y_predict_test=model_3.predict_proba(X_test)

roc_auc_score(y_train,y_predict_train,multi_class='ovo'),roc_auc_score(y_test,y_predict_test,multi_class='ovo')

(0.9495715107218877, 0.9359257995216238)

## The voting classifier

In [25]:
from sklearn.ensemble import VotingClassifier

multi_model=VotingClassifier(estimators=[('Kn',model_1_1),('gb',model_2),('Rf',model_3)],voting='soft')
multi_model.fit(X_train,y_train)

VotingClassifier(estimators=[('Kn',
                              BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=9),
                                                n_estimators=80)),
                             ('gb',
                              GradientBoostingClassifier(max_depth=6,
                                                         n_estimators=80)),
                             ('Rf',
                              RandomForestClassifier(max_depth=6,
                                                     n_estimators=50))],
                 voting='soft')

In [26]:
y_predict_train=multi_model.predict_proba(X_train)
y_predict_test=multi_model.predict_proba(X_test)

roc_auc_score(y_train,y_predict_train,multi_class='ovo'),roc_auc_score(y_test,y_predict_test,multi_class='ovo')

(0.9906779796118391, 0.9427701040495098)

## Feature Engineering (OneHotEncoding)

In [27]:
from sklearn.preprocessing import OneHotEncoder
Array=np.array(Train_final['Activity'])
Array=Array.reshape(-1,1)
coder=OneHotEncoder(sparse=False).fit(Array)
y=coder.transform(Array)


In [28]:
X=np.array(Train_final.loc[:,['fBodyAccJerk-entropy()-X', 'tBodyGyroJerk-entropy()-Z','tBodyAccJerk-entropy()-Y', 'tBodyAcc-sma()','tBodyAccJerkMag-entropy()', 'tBodyAcc-min()-X']])

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

## Building models:
We choose the KNeighborsClassifiers and RandomForestClassifer as they belong to the few algorithms that support multi-label classification, we cannot use votingClassifier as it's not implemented

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
scaler=StandardScaler()

X_train_scaled=scaler.fit_transform(X_train)

X_test_scaled=scaler.transform(X_test)

model=KNeighborsClassifier().fit(X_train_scaled,y_train)





param_grid={'n_neighbors': np.arange(5,30)}
model=GridSearchCV(model,param_grid,scoring='roc_auc')
model.fit(X_train_scaled,y_train)
model.best_params_,model.best_score_


({'n_neighbors': 29}, 0.9150436987509147)

In [31]:
model_1=KNeighborsClassifier(n_neighbors=29)
model_1.fit(X_train_scaled,y_train)
A_train=model_1.predict_proba(X_train_scaled)
b_train=A_train[0][:,1]
b_train=b_train.reshape(-1,1)

A_test=model_1.predict_proba(X_test_scaled)
b_test=A_test[0][:,1]
b_test=b_test.reshape(-1,1)
roc_auc_score(y_train,b_train),roc_auc_score(y_test,b_test)


(0.9118105305398327, 0.8777811363436918)

In [32]:
from sklearn.ensemble import RandomForestClassifier
param_grid={'n_estimators':np.arange(40,81,10), 'max_depth':np.arange(3,9)}
model=GridSearchCV(RandomForestClassifier(),param_grid,scoring='roc_auc')

model.fit(X_train,y_train)
model.best_params_, model.best_score_

({'max_depth': 8, 'n_estimators': 80}, 0.9353947627533632)

In [35]:
model_3=RandomForestClassifier(n_estimators=80, max_depth=8)
model_3.fit(X_train,y_train)
A_train=model_3.predict_proba(X_train)
b_train=A_train[0][:,1]
b_train=b_train.reshape(-1,1)

A_test=model_3.predict_proba(X_test)
b_test=A_test[0][:,1]
b_test=b_test.reshape(-1,1)
roc_auc_score(y_train,b_train),roc_auc_score(y_test,b_test)

(0.9629420461265524, 0.9246389812494515)